In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

import setting

path_dict = path(setting)

In [ ]:
w = pd.read_table(
    path_dict['w_file_path'],
    index_col=0,
)

h = pd.read_table(
    path_dict['h_file_path'],
    index_col=0,
)

In [ ]:
from numpy import absolute, where
from scipy.spatial.distance import correlation


def compute_correlation_distance(
        x,
        y,
):

    correlation_distance = correlation(
        x,
        y,
    )

    return where(
        absolute(correlation_distance) < 1e-8,
        0,
        correlation_distance,
    )

In [ ]:
for wt_or_h, df, bandwidth_factor, element_marker_size in (
    (
        'wt',
        w.T,
        setting.GPS_MAP_WT_BANDWIDTH_FACTOR,
        setting.GPS_MAP_WT_ELEMENT_MARKER_SIZE,
    ),
    (
        'h',
        h,
        setting.GPS_MAP_H_BANDWIDTH_FACTOR,
        setting.GPS_MAP_H_ELEMENT_MARKER_SIZE,
    ),
):
    
    distance__column_x_column, k_hcc_distance_linkage, k_hcc_distance_linkage_ccc, hcc__k_x_column = ccal.hierarchical_consensus_cluster_with_multiple_k(
        df,
        setting.HCC_KS,
        n_job=setting.MAX_N_JOB,
        distance_function=compute_correlation_distance,
        plot_df=False,
        directory_path=path_dict['{}_hcc_directory_path'.format(wt_or_h)],
    )